# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


In [2]:
from sklearn.model_selection import GridSearchCV


In [3]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk.download()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gayathriviswanathan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gayathriviswanathan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gayathriviswanathan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# load data from database
engine = create_engine('sqlite:///Disaster_Database.db')
df = pd.read_sql_table('Disaster_Msg',engine)
df.head()



,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2.0,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.0,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8.0,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,12.0,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df['message'] = df['message'].fillna('')  # Replace NaN with empty string
df.iloc[:,4:] = df.iloc[:,4:].fillna(0)
    
# Convert to string
df['message'] = df['message'].astype(str)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
        # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenization
    tokens = word_tokenize(text.lower())
    
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Remove very short tokens
    tokens = [token for token in tokens if len(token) > 1]
    
    return tokens

In [8]:
sample_text = "Disaster response is crucial for effective emergency management!"
processed_tokens = tokenize(sample_text)
print(processed_tokens)

['disaster', 'response', 'is', 'crucial', 'for', 'effective', 'emergency', 'management']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
pipeline = Pipeline([
        # TF-IDF Vectorization with custom tokenizer
        ('tfidf', TfidfVectorizer(
            tokenizer=tokenize,
            max_features=5000,
            ngram_range=(1, 2)
        )),
        # Multi-output classifier with RandomForest
        ('classifier', MultiOutputClassifier(
            RandomForestClassifier(
                n_estimators=100, 
                random_state=42, 
                n_jobs=-1
            )
        ))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [14]:

#sklearn.set_config(enable_metadata_routing=True)
pipeline.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=5000, ngram_range=(1, 2),
                                 tokenizer=<function tokenize at 0x12ffa6c20>)),
                ('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1,
                                                                        random_state=42)))])

In [15]:
print(X_train)

40820                                                      
75712                                                      
77426                                                      
74953                                                      
63260                                                      
                                ...                        
54886                                                      
76820                                                      
103694                                                     
860       We are in Fontamara on Louis St the route 1 an...
15795     The meteorological sector will take this good ...
Name: message, Length: 84104, dtype: object


In [16]:
print(y_train)

        related  request  offer  aid_related  medical_help  medical_products  \
40820       1.0      0.0    0.0          1.0           0.0               0.0   
75712       0.0      0.0    0.0          0.0           0.0               0.0   
77426       1.0      0.0    0.0          1.0           0.0               0.0   
74953       1.0      0.0    0.0          1.0           0.0               1.0   
63260       1.0      0.0    1.0          1.0           0.0               0.0   
...         ...      ...    ...          ...           ...               ...   
54886       1.0      1.0    0.0          1.0           0.0               0.0   
76820       0.0      0.0    0.0          0.0           0.0               0.0   
103694      1.0      0.0    0.0          1.0           0.0               0.0   
860         0.0      0.0    0.0          0.0           0.0               0.0   
15795       0.0      0.0    0.0          0.0           0.0               0.0   

        search_and_rescue  security  mi

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred = pipeline.predict(X_test)


In [18]:
def evaluate_model(y_test, y_pred, y_columns):
    # Comprehensive evaluation
    print("Detailed Classification Report:")
    for i, col in enumerate(y_columns):
        print(f"\nMetrics for {col}:")
        print(classification_report(y_test.iloc[:, i], y_pred[:, i]))

In [19]:
evaluate_model(y_test, y_pred, Y.columns)


Detailed Classification Report:

Metrics for related:
              precision    recall  f1-score   support

         0.0       1.00      0.58      0.74      8999
         1.0       0.75      1.00      0.86     11901
         2.0       0.00      0.00      0.00       126

    accuracy                           0.82     21026
   macro avg       0.58      0.53      0.53     21026
weighted avg       0.85      0.82      0.80     21026


Metrics for request:
              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93     18316
         1.0       0.00      0.00      0.00      2710

    accuracy                           0.87     21026
   macro avg       0.44      0.50      0.47     21026
weighted avg       0.76      0.87      0.81     21026


Metrics for offer:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20951
         1.0       0.00      0.00      0.00        75

    accuracy                    

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     20221
         1.0       0.00      0.00      0.00       805

    accuracy                           0.96     21026
   macro avg       0.48      0.50      0.49     21026
weighted avg       0.92      0.96      0.94     21026


Metrics for search_and_rescue:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     20614
         1.0       0.00      0.00      0.00       412

    accuracy                           0.98     21026
   macro avg       0.49      0.50      0.50     21026
weighted avg       0.96      0.98      0.97     21026


Metrics for security:
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     20743
         1.0       0.00      0.00      0.00       283

    accuracy                           0.99     21026
   macro avg       0.49      0.50      0.50     21026
weighted avg       

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     20782
         1.0       0.00      0.00      0.00       244

    accuracy                           0.99     21026
   macro avg       0.49      0.50      0.50     21026
weighted avg       0.98      0.99      0.98     21026


Metrics for money:
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     20666
         1.0       0.00      0.00      0.00       360

    accuracy                           0.98     21026
   macro avg       0.49      0.50      0.50     21026
weighted avg       0.97      0.98      0.97     21026


Metrics for missing_people:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     20840
         1.0       0.00      0.00      0.00       186

    accuracy                           0.99     21026
   macro avg       0.50      0.50      0.50     21026
weighted avg       0.98  

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     20683
         1.0       0.00      0.00      0.00       343

    accuracy                           0.98     21026
   macro avg       0.49      0.50      0.50     21026
weighted avg       0.97      0.98      0.98     21026


Metrics for tools:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     20939
         1.0       0.00      0.00      0.00        87

    accuracy                           1.00     21026
   macro avg       0.50      0.50      0.50     21026
weighted avg       0.99      1.00      0.99     21026


Metrics for hospitals:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     20828
         1.0       0.00      0.00      0.00       198

    accuracy                           0.99     21026
   macro avg       0.50      0.50      0.50     21026
weighted avg       0.98      0

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97     19765
         1.0       0.00      0.00      0.00      1261

    accuracy                           0.94     21026
   macro avg       0.47      0.50      0.48     21026
weighted avg       0.88      0.94      0.91     21026


Metrics for storm:
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.97     19607
         1.0       0.00      0.00      0.00      1419

    accuracy                           0.93     21026
   macro avg       0.47      0.50      0.48     21026
weighted avg       0.87      0.93      0.90     21026


Metrics for fire:
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     20838
         1.0       0.00      0.00      0.00       188

    accuracy                           0.99     21026
   macro avg       0.50      0.50      0.50     21026
weighted avg       0.98      0.99  

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [ ]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'tfidf', 'classifier', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'classifier__estimator__bootstrap', 'classifier__estimator__ccp_alpha', 'classifier__estimator__class_weight', 'classifier__estimator__criterion', 'classifier__estimator__max_depth', 'classifier__estimator__max_features', 'classifier__estimator__max_leaf_nodes', 'classifier__estimator__max_samples', 'classifier__estimator__min_impurity_decrease', 'classifier__estimator__min_samples_leaf', 'classifier__estimator__min_samples_split', 'classifier__estimator__min_weight_fraction_leaf', 'classifier

### 6. Improve your model
Use grid search to find better parameters. 

In [30]:
parameters =  { 
   'classifier__estimator__n_estimators': [25, 50,100,150],
   'classifier__estimator__max_depth': [3, 6, 9], 
    'classifier__estimator__max_leaf_nodes': [3, 6, 9], 
} 

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, scoring='accuracy', cv=2, n_jobs=-1)


In [31]:
cv.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/

GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=5000,
                                                        ngram_range=(1, 2),
                                                        tokenizer=<function tokenize at 0x12ffa6c20>)),
                                       ('classifier',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1,
                                                                                               random_state=42)))]),
             n_jobs=-1,
             param_grid={'classifier__estimator__max_depth': [3, 6, 9],
                         'classifier__estimator__max_leaf_nodes': [3, 6, 9],
                         'classifier__estimator__n_estimators': [25, 50, 100,
                                                                 150]},
             scoring='accuracy')

In [33]:
print("Best Hyperparameters:", cv.best_params_)


Best Hyperparameters: {'classifier__estimator__max_depth': 3, 'classifier__estimator__max_leaf_nodes': 3, 'classifier__estimator__n_estimators': 25}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.